# 04 — Tổng Hợp Visualization: Titans Memory Pipeline

Notebook này chạy toàn bộ pipeline trên nhiều scenario khác nhau để hiểu sâu hơn cách Titans hoạt động.

In [ ]:
import torch
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from titans_memory import (
    TitansMemoryLayer,
    generate_repeating_with_anomalies,
    generate_frequency_shift,
)

def run_and_plot_dashboard(seq, mask, title, momentum=0.9, forget_rate=0.01):
    """Chạy TitansMemoryLayer và vẽ dashboard 5 panel."""
    torch.manual_seed(42)
    layer = TitansMemoryLayer(
        input_dim=1, memory_dim=16, hidden_dim=16,
        momentum=momentum, forget_rate=forget_rate,
    )
    result = layer.process_sequence(seq.unsqueeze(-1))

    scores = [s.item() for s in result["surprise_scores"]]
    mom = [s.item() for s in result["surprise_momentum"]]
    anomaly_idx = mask.nonzero().squeeze(-1).numpy() if mask.any() else []

    fig = plt.figure(figsize=(14, 10))
    gs = gridspec.GridSpec(3, 2, hspace=0.4, wspace=0.3)

    ax1 = fig.add_subplot(gs[0, :])
    ax1.plot(seq.numpy(), color="#2196F3", linewidth=1)
    if len(anomaly_idx) > 0:
        ax1.scatter(anomaly_idx, seq[mask.bool()].numpy(), color="#F44336", s=80, zorder=5)
    ax1.set_title("Input Sequence")
    ax1.set_ylabel("Value")

    ax2 = fig.add_subplot(gs[1, 0])
    ax2.plot(scores, color="#FF9800", linewidth=1.5)
    ax2.set_title("Raw Surprise")
    ax2.set_ylabel("Score")

    ax3 = fig.add_subplot(gs[1, 1])
    ax3.plot(mom, color="#E91E63", linewidth=1.5)
    ax3.set_title("Momentum Surprise")
    ax3.set_ylabel("Score")

    snapshots = result["memory_snapshots"]
    first_layer = [s[0].numpy() for s in snapshots]
    changes = []
    for i in range(1, len(first_layer)):
        diff = np.abs(first_layer[i] - first_layer[i - 1])
        changes.append(diff.flatten()[:12])
    if changes:
        changes = np.array(changes).T
        ax4 = fig.add_subplot(gs[2, 0])
        ax4.imshow(changes, aspect="auto", cmap="YlOrRd", interpolation="nearest")
        ax4.set_title("Memory Update Intensity")
        ax4.set_xlabel("Timestep")
        ax4.set_ylabel("Weight Index")

    norms = [sum(w.norm().item() for w in s) for s in snapshots]
    ax5 = fig.add_subplot(gs[2, 1])
    ax5.plot(norms, color="#4CAF50", linewidth=2)
    ax5.set_title("Memory Weight Norm")
    ax5.set_xlabel("Timestep")
    ax5.set_ylabel("Norm")

    fig.suptitle(title, fontsize=14, fontweight="bold", y=0.98)
    plt.show()

## Scenario 1: Pattern lặp lại + Anomalies

Chuỗi [1,2,3] lặp lại 20 lần, với anomalies tại các vị trí 14, 35, 50.

In [ ]:
seq1, mask1 = generate_repeating_with_anomalies(
    pattern=[1.0, 2.0, 3.0], repeats=20,
    anomaly_indices=[14, 35, 50], anomaly_value=99.0,
)
run_and_plot_dashboard(seq1, mask1, "Scenario 1: Repeating Pattern + Anomalies")

## Scenario 2: Frequency Shift Signal

Sóng sin thay đổi tần số tại timestep 100 — liệu memory có phát hiện?

In [ ]:
seq2, shift_points = generate_frequency_shift(
    length=200, base_freq=0.05, shifted_freq=0.2, shift_at=[100]
)
mask2 = torch.zeros(200)
mask2[100] = 1.0  # Mark shift point
run_and_plot_dashboard(seq2, mask2, "Scenario 2: Frequency Shift at t=100")

## Scenario 3: So sánh các giá trị Momentum

Cùng dữ liệu, thay đổi momentum từ 0.0 đến 0.99.

In [ ]:
seq3, mask3 = generate_repeating_with_anomalies(
    pattern=[1.0, 2.0, 3.0, 4.0], repeats=15,
    anomaly_indices=[12, 30, 45], anomaly_value=80.0,
)

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
for ax, beta in zip(axes.flat, [0.0, 0.5, 0.9, 0.99]):
    torch.manual_seed(42)
    layer = TitansMemoryLayer(
        input_dim=1, memory_dim=16, hidden_dim=16,
        momentum=beta, forget_rate=0.01,
    )
    result = layer.process_sequence(seq3.unsqueeze(-1))
    scores = [s.item() for s in result["surprise_momentum"]]
    ax.plot(scores, color="#E91E63", linewidth=1.5)
    ax.set_title(f"Momentum = {beta}")
    ax.set_xlabel("Timestep")
    ax.set_ylabel("Surprise")

plt.suptitle("Ảnh hưởng của Momentum lên Surprise Response", fontsize=13, fontweight="bold")
plt.tight_layout()
plt.show()

## Scenario 4: So sánh Forget Rates

Cùng dữ liệu, thay đổi forget_rate từ 0.0 đến 0.1.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
for ax, rate in zip(axes.flat, [0.0, 0.005, 0.02, 0.1]):
    torch.manual_seed(42)
    layer = TitansMemoryLayer(
        input_dim=1, memory_dim=16, hidden_dim=16,
        momentum=0.9, forget_rate=rate,
    )
    result = layer.process_sequence(seq3.unsqueeze(-1))
    norms = [sum(w.norm().item() for w in s) for s in result["memory_snapshots"]]
    ax.plot(norms, color="#4CAF50", linewidth=2)
    ax.set_title(f"Forget Rate = {rate}")
    ax.set_xlabel("Timestep")
    ax.set_ylabel("Memory Norm")

plt.suptitle("Ảnh hưởng của Forget Rate lên Memory Persistence", fontsize=13, fontweight="bold")
plt.tight_layout()
plt.show()

## Tổng Kết

**Titans Surprise-Based Memory** hoạt động dựa trên 3 cơ chế chính:

1. **Surprise Metric** — Đo "độ bất ngờ" bằng prediction error
2. **Surprise-Gated Writes** — Chỉ ghi nhớ thông tin mới lạ (surprise cao)
3. **Adaptive Forgetting** — Tự động quên thông tin cũ qua weight decay

Kết hợp với **Momentum** để làm mịn tín hiệu surprise, hệ thống này cho phép AI:
- Xử lý chuỗi rất dài (>2M tokens) hiệu quả
- Ghi nhớ chọn lọc — chỉ giữ lại những gì quan trọng
- Kết hợp tốc độ RNN với độ chính xác Transformer